In [2]:
import psycopg2
import configparser
import pandas as pd

In [3]:
config = configparser.ConfigParser()
config.read("dwh.cfg")

['dwh.cfg']

In [4]:
conn = psycopg2.connect("host={} dbname={} user={} password={}".format(config["CLUSTER"]["HOST"],
                                                                config["IMMIGRATION"]["DB_NAME"],
                                                                config["CLUSTER"]["DB_USER"],
                                                                config["CLUSTER"]["DB_PASSWORD"]))

In [5]:
cursor = conn.cursor()

### Query 1
What is the population top 3 most love states by immigrants?

#### Building query 1
What are the states with the most immigration in US?

In [26]:
pd.read_sql_query('select im.i94addr as address, COUNT(i94addr) from immigration im GROUP BY im.i94addr'
                  ' ORDER BY COUNT(i94addr) DESC limit 3;', conn)


,address,count
0,FL,621701
1,NY,553677
2,CA,470386


#### Building query 2
What is the total population of the top 3 most loved states by immigrants?

In [20]:
pd.read_sql_query("""select population_by_state."state code", population_by_state.total_population from (select sc."state code", sum(sc."total population") as total_population from state_codes sc """ +
                  """group by sc."state code") as population_by_state;""", conn)

,state code,total_population
0,CA,123444353.0
1,NH,990990.0
2,OR,7182545.0
3,ND,947450.0
4,TX,70553853.0
5,PR,2030847.0
6,NV,11203720.0
7,OH,12096550.0
8,KY,4649385.0
9,NY,49002055.0


#### Building query final
What is the population top 3 most loved states by immigrants?

In [29]:
pd.read_sql_query("""select sum(population_by_state.total_population) from (select im.i94addr as address, COUNT(i94addr) from immigration im GROUP BY im.i94addr """ +
                  """ ORDER BY COUNT(i94addr) DESC limit 3) as top3 JOIN (select sc."state code", sum(sc."total population") """ +
                  """as total_population from state_codes sc group by sc."state code") as population_by_state ON """ +
                  """top3.address = population_by_state."state code" limit 5;""", conn)


,sum
0,204752540.0


### Query 2
What types of airports immigrants prefer?


In [39]:
#### Building query 2
How many immigrants used particular airport?
pd.read_sql_query("""select im.i94port port, count(im.i94port) count from immigration im group by im.i94port""" +
                  """ order by count desc limit 200""", conn)

,port,count
0,NYC,485916
1,MIA,343941
2,LOS,310163
3,SFR,152586
4,ORL,149195
...,...,...
195,PRO,12
196,ROM,12
197,PTK,12
198,MAF,11


In [44]:
## Building query final
What types of airports immigrants prefer?

pd.read_sql_query("""select ac.type, sum(count_by_port.count) from (select im.i94port port, count(im.i94port) count from immigration im group by im.i94port""" +
                  """ order by count desc limit 200) as count_by_port join airport_codes ac on count_by_port.port = ac.iata_code""" +
                  """ group by ac.type limit 10""", conn)

,type,sum
0,closed,161150.0
1,large_airport,1174221.0
2,medium_airport,410990.0
3,seaplane_base,659.0
4,small_airport,288700.0
